In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)

import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
import glob

target_size = (32, 32)  # Change the values as per your requirement

# Load the pre-trained ResNet50 model with modified input shape
model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(target_size[0], target_size[1], 3))

In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"G:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [04:25<00:00, 13.97s/it]

X_train shape: (2872, 32, 32, 3)
Y_train shape: (2872,)


In [2]:
import pandas as pd
# Define the file path
file_path = r"D:\lbpfeature.csv"
# Read the tab-separated CSV file into a DataFrame
df = pd.read_csv(file_path, delimiter='\t')
columns_to_drop = [df.columns[0], df.columns[-1]]
data = df.drop(columns_to_drop, axis=1)
# Display the head of the DataFrame
data.head(1)

,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,...,Feature 246,Feature 247,Feature 248,Feature 249,Feature 250,Feature 251,Feature 252,Feature 253,Feature 254,Feature 255
0,0.625651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.329194,...,4.020732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from tensorflow.keras.datasets import cifar10
from sklearn.cluster import MiniBatchKMeans
from PIL import Image
import glob
from tqdm import tqdm

# Parameters for LBP
radius = 3
n_points = 8 * radius  # Adjusted to ensure 24 points for LBP
METHOD = 'uniform'
num_bins = 256  # Explicitly set bins to 256 for the histogram

# Function to compute LBP histograms for a list of images
def compute_lbp_histogram(images, num_bins):
    histograms = []
    for image in images:
        image = np.array(image.resize((32, 32))).astype("float32") / 255.0
        image_gray = rgb2gray(image)
        lbp = local_binary_pattern(image_gray, n_points, radius, METHOD)
        hist, _ = np.histogram(lbp, density=True, bins=num_bins, range=(0, n_points + 2))
        histograms.append(hist)
    return np.array(histograms)

lbp_features = compute_lbp_histogram([Image.fromarray(img) for img in X_train], num_bins)

# Parameters for MiniBatchKMeans
n_clusters = 400
batch_size = 100
max_iter = 100

# Apply MiniBatchKMeans clustering
kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, max_iter=max_iter)
kmeans.fit(lbp_features)

# Retrieve the cluster centers
cluster_centers = kmeans.cluster_centers_.tolist()

print(cluster_centers)


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


[[0.6025641025641026, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4679487179487179, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26282051282051283, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1490384615384615, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1266025641025641, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08814102564102563, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0705128205128205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10737179487179485, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10737179487179485, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08012820512820512, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07211538461538461, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08173076923076923, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1217948717948718, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2548076923076923, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09455128205128205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18269230769230765, 0.0, 0.0,

In [4]:
from tqdm import tqdm
ct = cluster_centers
combo_list = []  # Initialize combo_list
tot_dist = []
# Iterate over ft
for i in tqdm(range(len(ct))):
    distances = []

    # Calculate distances for each row in data
    for index, row in data.iterrows():
        row_array = row.to_numpy()  # Convert row to numpy array
        distance = np.linalg.norm(ct[i] - row_array)  # Calculate Euclidean distance
        distances.append(distance)

    tot_dist.append(distances)
print(np.shape(tot_dist))
#new_shape = (1000, 30)
#re_dist = np.transpose(tot_dist, (1, 0))
re_dist = tot_dist
print(np.shape(re_dist))

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:29<00:00, 13.53it/s]

(400, 918)
(400, 918)


In [6]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\dist_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(re_dist, file)

print("Combo list saved successfully.")

Combo list saved successfully.


In [8]:
import pickle

file_path = r"D:\index_list.pickle" 

with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")


Combo list loaded successfully.


In [9]:
combo_list = df.loc[loaded_combo_list, 'Directory'].tolist()
combo_list

['D:\\data\\imagenet\\n01484850\\',
 'D:\\data\\imagenet\\n01491361\\',
 'D:\\data\\imagenet\\n01494475\\',
 'D:\\data\\imagenet\\n01514859\\',
 'D:\\data\\imagenet\\n01530575\\',
 'D:\\data\\imagenet\\n01534433\\',
 'D:\\data\\imagenet\\n01537544\\',
 'D:\\data\\imagenet\\n01558993\\',
 'D:\\data\\imagenet\\n01560419\\',
 'D:\\data\\imagenet\\n01580077\\',
 'D:\\data\\imagenet\\n01582220\\',
 'D:\\data\\imagenet\\n01601694\\',
 'D:\\data\\imagenet\\n01614925\\',
 'D:\\data\\imagenet\\n01622779\\',
 'D:\\data\\imagenet\\n01629819\\',
 'D:\\data\\imagenet\\n01630670\\',
 'D:\\data\\imagenet\\n01631663\\',
 'D:\\data\\imagenet\\n01632458\\',
 'D:\\data\\imagenet\\n01641577\\',
 'D:\\data\\imagenet\\n01644373\\',
 'D:\\data\\imagenet\\n01644900\\',
 'D:\\data\\imagenet\\n01667114\\',
 'D:\\data\\imagenet\\n01669191\\',
 'D:\\data\\imagenet\\n01675722\\',
 'D:\\data\\imagenet\\n01677366\\',
 'D:\\data\\imagenet\\n01685808\\',
 'D:\\data\\imagenet\\n01687978\\',
 'D:\\data\\imagenet\\n01688

In [10]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\lbp_cifar_combo_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(combo_list, file)

print("Combo list saved successfully.")


Combo list saved successfully.


In [11]:
import pickle

# File path of the saved pickle
file_path =  r"D:\lbp_cifar_combo_list.pickle"

# Load the data from the pickle file
with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")


Combo list loaded successfully.


In [12]:
loaded_combo_list

['D:\\data\\imagenet\\n01484850\\',
 'D:\\data\\imagenet\\n01491361\\',
 'D:\\data\\imagenet\\n01494475\\',
 'D:\\data\\imagenet\\n01514859\\',
 'D:\\data\\imagenet\\n01530575\\',
 'D:\\data\\imagenet\\n01534433\\',
 'D:\\data\\imagenet\\n01537544\\',
 'D:\\data\\imagenet\\n01558993\\',
 'D:\\data\\imagenet\\n01560419\\',
 'D:\\data\\imagenet\\n01580077\\',
 'D:\\data\\imagenet\\n01582220\\',
 'D:\\data\\imagenet\\n01601694\\',
 'D:\\data\\imagenet\\n01614925\\',
 'D:\\data\\imagenet\\n01622779\\',
 'D:\\data\\imagenet\\n01629819\\',
 'D:\\data\\imagenet\\n01630670\\',
 'D:\\data\\imagenet\\n01631663\\',
 'D:\\data\\imagenet\\n01632458\\',
 'D:\\data\\imagenet\\n01641577\\',
 'D:\\data\\imagenet\\n01644373\\',
 'D:\\data\\imagenet\\n01644900\\',
 'D:\\data\\imagenet\\n01667114\\',
 'D:\\data\\imagenet\\n01669191\\',
 'D:\\data\\imagenet\\n01675722\\',
 'D:\\data\\imagenet\\n01677366\\',
 'D:\\data\\imagenet\\n01685808\\',
 'D:\\data\\imagenet\\n01687978\\',
 'D:\\data\\imagenet\\n01688